In [ ]:
import pickle

# Save the model using pickle
with open('word2vec_model.pkl', 'wb') as f:
    pickle.dump(word2vec_model, f)

from gensim.models import KeyedVectors

# Load the saved Word2Vec model
word2vec_model = KeyedVectors.load('/content/word2vec_model.pkl')


import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
# import 

# Get the vocabulary from the pre-trained model
vocabulary = word2vec_model.wv.vocab

# Fine-tune Word2Vec model using your data
filtered_sentences = [[word for word in word_tokenize(description) if word in vocabulary] 
                      for description in filtered_train_df['bug_description']]


# Create and train the Word2Vec model
fine_tuned_word2vec_model = Word2Vec(size=300, min_count=1)
fine_tuned_word2vec_model.build_vocab(filtered_sentences, update=True)
fine_tuned_word2vec_model.train(filtered_sentences, total_examples=len(filtered_sentences), epochs=10)

# Function to get average word vector for a description
def get_average_word_vector(description, word_embeddings_model, embedding_size):
    words = word_tokenize(description)
    vectors = [word_embeddings_model[word] for word in words if word in word_embeddings_model]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(embedding_size)  # Return zero vector if no words are found in the model

# Tokenize and get document vectors for training data
train_embeddings = np.array([get_average_word_vector(description, fine_tuned_word2vec_model, 300) 
                             for description in filtered_train_df['bug_description']])

# Tokenize and get document vectors for testing data
test_embeddings = np.array([get_average_word_vector(description, fine_tuned_word2vec_model, 300) 
                            for description in filtered_test_df['bug_description']])

# Perform clustering (K-means) on training data
num_clusters = len(class_name_mapping)
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
train_clusters = kmeans.fit_predict(train_embeddings)

# Train SVM model with tuned hyperparameters
svm_model = SVC(kernel='linear', C=100)
svm_model.fit(train_embeddings, train_clusters)

# Make predictions on the entire test set
test_clusters_pred = svm_model.predict(test_embeddings)

# Define a mapping function to map labels
def map_labels(label):
    if label == 'Frontend':
        return 1
    elif label == 'Backend':
        return 0
    else:
        return None

# Apply the mapping function to filtered_test_df['class_name']
mapped_labels = filtered_test_df['class_name'].map(map_labels)

# Calculate accuracy with mapped labels
accuracy = accuracy_score(mapped_labels, test_clusters_pred)
print("Accuracy of SVM model:", accuracy)
